<a href="https://colab.research.google.com/github/smitdesai1010/COVID-19-Detection/blob/master/COVID_19_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"smitdesai1010","key":"41f928f2f4a0f0b462ed2ec45f383c82"}'}

In [ ]:
import json

token = {"username":"smitdesai1010","key":"41f928f2f4a0f0b462ed2ec45f383c82"}
with open('/content/kaggle.json', 'w') as file:
    json.dump(token, file)

!cp /content/kaggle.json ~/.kaggle/kaggle.json

!kaggle config set -n path -v /content
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d amanullahasraf/covid19-pneumonia-normal-chest-xray-pa-dataset

- path is now set to: /content
100% 1.91G/1.91G [00:12<00:00, 111MB/s]
100% 1.91G/1.91G [00:12<00:00, 169MB/s]


In [ ]:
import zipfile

zf = zipfile.ZipFile('/content/datasets/amanullahasraf/covid19-pneumonia-normal-chest-xray-pa-dataset/covid19-pneumonia-normal-chest-xray-pa-dataset.zip') 
zf.extractall()

!rm -rf 'datasets' #Deleting the the downloaded Zip files after extracting it 

In [ ]:
import tensorflow as tf
import cv2
import os
import datetime

import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def create_dataset(dir):
  path = '/content/'+dir+'/'

  for file in os.listdir(path):

    img = cv2.imread(path+file , cv2.IMREAD_COLOR)
    
    try:
     img = cv2.resize(img,(1024,1024))

    except:
     continue 

    images.append(np.array(img))     
    labels.append(dir)  

https://medium.com/@mrgarg.rajat/training-on-large-datasets-that-dont-fit-in-memory-in-keras-60a974785d71

In [ ]:
images = []
labels = []

create_dataset('covid')
create_dataset('normal')
create_dataset('pneumonia') 

print(len(images))
print(len(labels))